In [1]:

import os
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
root_path = '/content/gdrive/My Drive/handin3/datasetfinal'  #change dir to your project folder
os.chdir(root_path)
os.getcwd()

ModuleNotFoundError: No module named 'google'

Import all relevant packages

In [ ]:
import numpy as np

import matplotlib.pyplot as plt 
#from skimage.io import imread
import glob
import os
from keras.callbacks import ModelCheckpoint
from keras import backend as K


In [ ]:
##–– Accessing the images in the Drive ––##
folders = glob.glob(root_path)
imagenames_list1 = []
imagenames_list2 = []
imagenames_list3 = []
imagenames_list4ff = []
imagenames_list5ff = []
imagenames_list6ff = []
for patch in sorted(glob.glob(root_path+'/m*.tif*')):
  imagenames_list1.append(patch)
for patch in sorted(glob.glob(root_path+'/p*.tif*')):
  imagenames_list2.append(patch)
for patch in sorted(glob.glob(root_path+'/c*.tif*')):
  imagenames_list3.append(patch)
#for patch in glob.glob(root_path+'/p*.tiff'):

  #imagenames_list4ff.append(patch)
#for patch in glob.glob(root_path+'/m*.tiff'):
  #imagenames_list5ff.append(patch)
#for patch in glob.glob(root_path+'/c*.tiff'):
  #imagenames_list6ff.append(patch)


  

In [ ]:
!pip install rasterio
import rasterio
import numpy as  np
X = np.zeros((97, 256, 256, 4))
y = np.zeros((97, 256, 256))
for count, image in enumerate(imagenames_list2):
    with rasterio.open(image, 'r') as src:
        data = src.read()
        data = np.swapaxes(data,0,1)
        data = np.swapaxes(data,1,2)
        X[count, :, :, 0:3] = data[:, :, 0:3]
        count += 1
for count,image in enumerate(imagenames_list3):
    with rasterio.open(image, 'r') as src:
        data = src.read()
        data = data[0,:, :]
        unique,counts=np.unique(data,return_counts=True)
        if counts[0] > counts[1]:
                data[data[:,:] == unique[0]] = 0
                data[data[:,:] == unique[1]]  = 1
        else:
                data[data[:,:] == unique[0]] = 1
                data[data[:,:] == unique[1]]  = 0
        if data.shape[0] == 250:
              data = np.full((256,256), 0)
        X[count, :, :, 3] = data
for count, image in enumerate(imagenames_list1):
    with rasterio.open(image, 'r')as src:
        data  = src.read()
        data = data[0, :, :]
        data = data.astype(np.float32)
        unique,counts=np.unique(data,return_counts=True)
        args = counts.argsort()[-2:][::-1]
        data[data[:, :] != unique[args[1]]] = np.nan
        data[data[:, :] == unique[args[1]]] = 1
        data[np.isnan(data[:, :])] = 0
        y[count, :, :] = data


In [ ]:
def normalize_data(X_train,X_test):
    for band in range(X_train.shape[3]):
        X_min = np.min(X_train[:,:,:,band])
        X_max = np.max(X_train[:,:,:,band])
        X_train[:,:,:,band] = (X_train[:,:,:,band] - X_min) / (X_max-X_min)
        X_test[:,:,:,band] = (X_test[:,:,:,band] - X_min) / (X_max-X_min)
    return X_train, X_test


In [ ]:
!pip install segmentation_models
import segmentation_models as sm


In [ ]:
from sklearn.model_selection import RepeatedKFold, cross_val_score, KFold
from keras.callbacks import ModelCheckpoint
from segmentation_models.losses  import bce_jaccard_loss
from segmentation_models.metrics import IOUScore
#filepath='/tmp/maxvals.hdf5'
filepath2='/tmp/loss.hdf5'
#kron=ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
kron2=ModelCheckpoint(filepath2, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max')
callbacks_list = [kron2]
                 
BACKBONE = 'vgg16'
model = sm.Unet(BACKBONE, encoder_weights=None, input_shape=(None, None, 4))
model.compile(
    'Adam',
    loss=sm.losses.categorical_crossentropy,
    metrics=['accuracy' ],
)

In [ ]:
import segmentation_models as sm
from segmentation_models.losses  import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from keras.optimizers import Adam
BACKBONE = 'resnet34'
filepath23='/tmp/loss2.hdf5'
np.random.seed(2)
foldsXX = []
foldsYY = []
iou_scores = []
X = X[:, :, :, 0:3]
for seed in np.random.randint(1000, size=2):
        np.random.seed(seed)
        idx = np.random.permutation(len(X))
        X,y = X[idx], y[idx]
        foldsX = []
        foldsY = []
        for fold in range(5):
            start = int((len(X)/5)) * fold
            end = int((len(X)/5)) * (fold+1)
            foldsX.append(X[start:end, :, :, :])
            foldsY.append(y[start:end, :, :])
    foldsXX.append(foldsX)
    foldsYY.append(foldsY)
    for fold in range(5):
        X_val = foldsX[fold]
        y_val = foldsY[fold]
        X_train = np.zeros((76, 256, 256, 3))
        y_train = np.zeros((76, 256, 256,1))
        counter = 0
    for fold_train in range(5):
        if fold_train != fold:
            X_train[(19*counter):(19*(counter+1)), :, :, :] = foldsX[fold_train]
            y_train[(19*counter):(19*(counter+1)), :, :] = foldsY[fold_train]
    #X_train, X_val = normalize_data(X_train, X_val)
    y_train = y_train.reshape((76, 256, 256, 1))
    y_val = y_val.reshape((19, 256, 256, 1))
    model = sm.Unet(BACKBONE, encoder_weights=None, input_shape=(None, None, 3))
    adam = Adam(lr=0.00001)
    model.compile(
        'Adam',
        loss=bce_jaccard_loss,
        metrics=[iou_score],
    )
    checkpoint = ModelCheckpoint(filepath23, monitor='val_iou_score', verbose=1, save_best_only=True, mode='max')
    model.fit(X_train,y_train, batch_size=32, epochs=100, validation_data=(X_val, y_val), callbacks=[checkpoint])
    model.load_weights(filepath23)
    print(model.evaluate(X_val, y_val)[1])
    iou_scores.append(model.evaluate(X_val, y_val)[1])
    # prwtoi 5 std kai mean  g t  prwto klp 
    

    
  
  

In [ ]:
print(model.evaluate(X_val, y_val)[1])
iou_scores.append(model.evaluate(X_val, y_val)[1])
print(iou_scores)

In [ ]:
print(model.evaluate(X_val, y_val)[1])
iou_scores.append(model.evaluate(X_val, y_val)[1])
print(iou_scores)
ion=np.asarray(iou_scores)
print(ion)
print(ion.shape)

       
        


In [ ]:
import segmentation_models as sm
from segmentation_models.losses  import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from keras.optimizers import Adam
BACKBONE = 'resnet34'
filepath23='/tmp/loss2.hdf5'

from sklearn.model_selection import train_test_split
y = y.reshape((97, 256, 256, 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#X_train, X_test = normalize_data(X_train, X_test)
model = sm.Unet(BACKBONE, encoder_weights=None, input_shape=(None, None, 4))
adam = Adam(lr=0.00001)
model.compile(
        'Adam',
        loss=bce_jaccard_loss,
        metrics=[iou_score],
             )
checkpoint = ModelCheckpoint(filepath23, monitor='val_iou_score', verbose=1, save_best_only=True, mode='max')
model.fit(X_train,y_train, batch_size=32, epochs=200, validation_data=(X_test, y_test), callbacks=[checkpoint])
load.model('loss2.hdf5')
score=model.evaluate(X_test, y_test,verbose=1)
ymo=model.predict(X_test,batch_size=32)
print(score)
	

In [ ]:
model.load_weights(filepath23)
score=model.evaluate(X_test, y_test,verbose=1)
ymo=model.predict(X_test,batch_size=32)
print(score)

In [ ]:
#plt.imshow(X[80,:,:,0])
#plt.imshow(y[1,:,:,0])
#print(ymo.shape)
y_pred = ymo > 0.49
plt.imshow(y_pred[1,:,:,0])
#1,29,50,80

# 2) Based on the training set: Describe the classifier, how it was trained and applied, and compute the variance and mean of the cross-validation classification performance (do e.g. 10 times randomized 5-fold cross-validation) as well as the true generalization performance on a completely held out data

# 3) Use the annotated data to build a CNN classifier (e.g. Unet) for counting cells in all the full image. 

For inspiration look at ttps://www.robots.ox.ac.uk/~vgg/publications/2015/Xie15/weidi15.pdf
Hint: To implement this, one can either build end-to-end CNNs for counting the number of cells or implement a cell counting method and apply it to the segmented images obtained from the first two parts.





Define a CNN (e.g. Unet) with its architecture

In [ ]:
##–– XXX-Net Architecture ––##
from scipy import ndimage
from scipy.ndimage.measurements import label
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
y = y.reshape((97, 256, 256, 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
BACKBONE = 'resnet34'
filepath3='/tmp/loss3.hdf5'

def exComp(thresholdedArray):
    ret, labels = cv2.connectedComponents(thresholdedArray)
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    
    components = cv2.merge([label_hue, blank_ch, blank_ch])
    
    #print(np.max(labels))
    #print(np.max(labels)-1)
    # set bg label to black
    components[label_hue==0] = 0
    k=np.amax(labels)
      
    ind = np.where(labels== 1)
    print(' len1 = ', len(ind[0]))
    ind2 = np.where(labels== 2)
    print(' len2 = ', len(ind2[0]))
    maxComponents = np.zeros(thresholdedArray.shape)
    maxComponents[ind] = 1
    maxComponents[ind2] = 2
    plt.imshow(maxComponents)
   # m= maxComponents[ind]+ maxComponents[ind2]
    print(k)
    return k
  
model = sm.Unet(BACKBONE, encoder_weights=None, input_shape=(None, None, 4))
adam = Adam(lr=0.00001)
model.compile(
         'Adam',
         loss=bce_jaccard_loss,
         metrics=[iou_score],
             )
checkpoint = ModelCheckpoint(filepath3, monitor='val_iou_score', verbose=1, save_best_only=True, mode='max')
model.fit(X_train,y_train, batch_size=32, epochs=200, validation_data=(X_test, y_test), callbacks=[checkpoint])
model.load_weights(filepath3)
ym=model.predict(X_test,batch_size=32)

nm=np.zeros(97)
for k in range(97):
    for i in range (256):
        for j in range (256):
            if ym[k,i,j,0]<0.5:
                ym[k,i,j,0]=0
            else:
            ym[k,i,j,0]=1

    labelarray, nm[k] = ndimage.measurements.label((ym[k,:,:,0]))
    ind = np.where(labelarray== 1)
    print(' len1 = ', len(ind[0]))
    ind2 = np.where(labelarray== 2)
    print(' len2 = ', len(ind2[0]))
    maxComponents = np.zeros(labelarray.shape)
    maxComponents[ind] = 1
    maxComponents[ind2] = 2
     plt.imshow(maxComponents)
    
print(nm)
        

In [ ]:
#plt.imshow(ym[96,:,:,0])

print(ym[96,:,:,0])
for i in range (256):
  for j in range (256):
    if ym[96,i,j,0]<0.02:
      ym[96,i,j,0]=0
    else:
      ym[96,i,j,0]=1
plt.imshow(ym[96,:,:,0])
labelarray, particle_count = ndimage.measurements.label((ym[96,:,:,0]))
#labelarray, particle_count = ndimage.measurements.label((ym[5,:,:,0]))
im=cv2.imread(plt.imshow(ym[5,:,:,0]))
plt.imshow(im)
ret, thresh = cv2.threshold(im, 127, 255, 0)

#---- Inverting the image here ----
img = cv2.bitwise_not(thresh) 

labelarray, particle_count = ndimage.measurements.label((img))

In [ ]:
print(particle_count)

Train the model on the training data

In [ ]:
##–– Training the model ––##
from scipy import ndimage
from scipy.ndimage.measurements import label
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from keras.optimizers import Adam
BACKBONE = 'resnet34'
filepath23='/tmp/loss2.hdf5'
np.random.seed(10)
foldsXX = []
foldsYY = []
iou_scores = []
X = X[:, :, :, 0:3]
for seed in np.random.randint(1000, size=10):
  np.random.seed(seed)
  idx = np.random.permutation(len(X))
  X,y = X[idx], y[idx]
  foldsX = []
  foldsY = []
  for fold in range(5):
    start = int((len(X)/5)) * fold
    end = int((len(X)/5)) * (fold+1)
    foldsX.append(X[start:end, :, :, :])
    foldsY.append(y[start:end, :, :])
  foldsXX.append(foldsX)
  foldsYY.append(foldsY)
  for fold in range(5):
    X_val = foldsX[fold]
    y_val = foldsY[fold]
    X_train = np.zeros((76, 256, 256, 3))
    y_train = np.zeros((76, 256, 256))
    counter = 0
    for fold_train in range(5):
      if fold_train != fold:
        X_train[(19*counter):(19*(counter+1)), :, :, :] = foldsX[fold_train]
        y_train[(19*counter):(19*(counter+1)), :, :] = foldsY[fold_train]  
    y_train = y_train.reshape((76, 256, 256, 1))
    y_val = y_val.reshape((19, 256, 256, 1))
    model = sm.Unet(BACKBONE, encoder_weights=None, input_shape=(None, None, 3))
    adam = Adam(lr=0.00001)
    model.compile(
        'Adam',
        loss=bce_jaccard_loss,
        metrics=[iou_score],
    )
    checkpoint = ModelCheckpoint(filepath23, monitor='val_iou_score', verbose=1, save_best_only=True, mode='max')
    model.fit(X_train,y_train, batch_size=8, epochs=50, validation_data=(X_val, y_val), callbacks=[checkpoint])
    model.load_weights(filepath23)
    #print(model.evaluate(X_val, y_val)[1])
    iou_scores.append(model.evaluate(X_val, y_val)[1])
    
   

In [ ]:
from sklearn.model_selection import train_test_split
#y = y.reshape((97, 256, 256, 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = sm.Unet(BACKBONE, encoder_weights=None, input_shape=(None, None, 3))
    adam = Adam(lr=0.00001)
    model.compile(
        'Adam',
        loss=bce_jaccard_loss,
        metrics=[iou_score],
    )
    checkpoint = ModelCheckpoint(filepath23, monitor='val_iou_score', verbose=1, save_best_only=True, mode='max')
    model.fit(X_train,y_train, batch_size=8, epochs=100, validation_data=(X_val, y_val), callbacks=[checkpoint])
ym=model.predict(X,batch_size=8)
nm=np.zeros(97)
for k in range(97):
  for i in range (256):
    for j in range (256):
      if ym[k,i,j,0]<0.02:
        ym[k,i,j,0]=0
      else:
        ym[k,i,j,0]=1

  labelarray, nm[k] = ndimage.measurements.label((ym[k,:,:,0]))
      
    
print(nm)
        